In [ ]:
!pip install python-box

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 3.0 MB 2.1 MB/s 


In [ ]:
import math
import numpy as np
import pandas as pd
from tqdm import tqdm
from collections import defaultdict
import os

import random
from datetime import datetime
from time import time
import scipy.sparse as sp

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from box import Box

import warnings

warnings.filterwarnings(action='ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/data

/content/drive/MyDrive/data


In [ ]:
data = pd.read_csv('/content/drive/MyDrive/data/total_menu_final_category_property.csv', index_col = 0)
data.head()

,Unnamed: 0.1,user_id,menu,menu_id,timestamp,category,property
0,0,0,찐빵,0,1,양식,빵류
1,1,0,맑은국,16,1,한식,국류
2,2,0,오징어찌개,1,2,한식,찌개류
3,3,0,육개장,2,3,한식,면류
4,4,0,단호박샌드,3,4,양식,빵류


In [ ]:
del data['Unnamed: 0.1']
data.head()

,user_id,menu,menu_id,timestamp,category,property
0,0,찐빵,0,1,양식,빵류
1,0,맑은국,16,1,한식,국류
2,0,오징어찌개,1,2,한식,찌개류
3,0,육개장,2,3,한식,면류
4,0,단호박샌드,3,4,양식,빵류


In [ ]:
# session별 history
history = data.groupby('user_id').menu.apply(list)
history.head()

user_id
0    [찐빵, 맑은국, 오징어찌개, 육개장, 단호박샌드, 김치찌개, 어묵국, 베이글, 팽...
1    [근대된장국, 인절미토스트, 우동, 유부장국, 순두부찌개, 씨크립샌드, 대구찌개, ...
2    [베이컨샌드, 사골우거지국, 사과파이, 양송이스프, 부대찌개, 오징어국, 브라우니,...
3    [어묵국, 단팥빵, 매운콩나물국, 크로와상, 잔치국수, 시금치국, 닭곰탕, 마늘빵,...
4    [수제비국, 팽이버섯장국, 크로와상, 냉이된장찌개, 야채샌드, 들깨미역국, 버섯들깨...
Name: menu, dtype: object

In [ ]:
#menu['timestamp'] = list(range(1, 31))*4518

In [ ]:
# 컬럼명 변경
data.rename(columns={'user_id':'session', 'menu':'item'}, inplace=True)
data.head()

,session,item,menu_id,timestamp,category,property
0,0,찐빵,0,1,양식,빵류
1,0,맑은국,16,1,한식,국류
2,0,오징어찌개,1,2,한식,찌개류
3,0,육개장,2,3,한식,면류
4,0,단호박샌드,3,4,양식,빵류


In [ ]:
print(menu.shape,menu["SessionID"].nunique())

(135540, 4) 4518


In [ ]:
# 새롭게 저장
data.to_csv("dacon_menu_sj.csv")

1. 학습 설정

In [ ]:
config = {
    'data_path' : "/content/drive/MyDrive/data" , # 데이터 경로

    'model_path' : "/content/drive/MyDrive/data", # 모델 저장 경로
    'model_name' : 'NGCF.pt',

    'num_epochs' : 50,
    "reg" : 1e-5,
    'lr' : 0.0001,
    "emb_dim" : 128,
    "layers" : [128, 128],
    'batch_size' : 500,
    "node_dropout" : 0.2,
    "mess_dropout" : 0.2,

    'valid_samples' : 10, # 검증에 사용할 sample 수
    'seed' : 22,
    'n_batch' : 10,
}

device = 'cuda' if torch.cuda.is_available() else 'cpu'

config = Box(config)

In [ ]:
if not os.path.isdir(config.model_path):
    os.mkdir(config.model_path)

2. 데이터 전처리

In [ ]:
class MakeGraphDataSet():
    """
    GraphDataSet 생성
    """
    def __init__(self, config):
        self.config = config
        self.df = pd.read_csv(os.path.join(self.config.data_path, "dacon_menu_sj.csv"))

        self.item_encoder, self.item_decoder = self.generate_encoder_decoder('menu_id')
        self.user_encoder, self.user_decoder = self.generate_encoder_decoder('session')
        self.num_item, self.num_user = len(self.item_encoder), len(self.user_encoder)

        self.df['item_idx'] = self.df['menu_id'].apply(lambda x : self.item_encoder[x])
        self.df['user_idx'] = self.df['session'].apply(lambda x : self.user_encoder[x])
        
        self.exist_users = [i for i in range(self.num_user)]
        self.exist_items = [i for i in range(self.num_item)]
        self.user_train, self.user_valid = self.generate_sequence_data()
        self.R_train, self.R_valid, self.R_total = self.generate_dok_matrix()
        self.ngcf_adj_matrix = self.generate_ngcf_adj_matrix()
        self.n_train = len(self.R_train)
        self.batch_size = self.config.batch_size

    def generate_encoder_decoder(self, col : str) -> dict:
        """
        encoder, decoder 생성

        Args:
            col (str): 생성할 columns 명
        Returns:
            dict: 생성된 user encoder, decoder
        """

        encoder = {}
        decoder = {}
        ids = self.df[col].unique()

        for idx, _id in enumerate(ids):
            encoder[_id] = idx
            decoder[idx] = _id

        return encoder, decoder
    
    def generate_sequence_data(self) -> dict:
        """
        sequence_data 생성

        Returns:
            dict: train user sequence / valid user sequence
        """
        users = defaultdict(list)
        user_train = {}
        user_valid = {}
        for user, item, time in zip(self.df['user_idx'], self.df['item_idx'], self.df['timestamp']):
            users[user].append(item)
        
        for user in users:
            np.random.seed(self.config.seed)

            user_total = users[user]
            valid = np.random.choice(user_total, size = self.config.valid_samples, replace = False).tolist()
            train = list(set(user_total) - set(valid))

            user_train[user] = train
            user_valid[user] = valid # valid_samples 개수 만큼 검증에 활용 (현재 Task와 가장 유사하게)

        return user_train, user_valid
    
    def generate_dok_matrix(self):
        R_train = sp.dok_matrix((self.num_user, self.num_item), dtype=np.float32)
        R_valid = sp.dok_matrix((self.num_user, self.num_item), dtype=np.float32)
        R_total = sp.dok_matrix((self.num_user, self.num_item), dtype=np.float32)
        user_list = self.exist_users
        for user in user_list:
            train_items = self.user_train[user]
            valid_items = self.user_valid[user]
            
            for train_item in train_items:
                R_train[user, train_item] = 1.0
                R_total[user, train_item] = 1.0
            
            for valid_item in valid_items:
                R_valid[user, valid_item] = 1.0
                R_total[user, valid_item] = 1.0
        
        return R_train, R_valid, R_total

    def generate_ngcf_adj_matrix(self):
        adj_mat = sp.dok_matrix((self.num_user + self.num_item, self.num_user + self.num_item), dtype=np.float32)
        adj_mat = adj_mat.tolil() # to_list
        R = self.R_train.tolil()

        adj_mat[:self.num_user, self.num_user:] = R
        adj_mat[self.num_user:, :self.num_user] = R.T
        adj_mat = adj_mat.todok() # to_dok_matrix

        def normalized_adj_single(adj):
            rowsum = np.array(adj.sum(1))
            d_inv = np.power(rowsum, -.5).flatten()  
            d_inv[np.isinf(d_inv)] = 0.
            d_mat_inv = sp.diags(d_inv)
            norm_adj = d_mat_inv.dot(adj).dot(d_mat_inv)

            return norm_adj.tocoo()

        ngcf_adj_matrix = normalized_adj_single(adj_mat)
        return ngcf_adj_matrix.tocsr()

    def sampling(self):
        users = random.sample(self.exist_users, self.config.batch_size)

        def sample_pos_items_for_u(u, num):
            pos_items = self.user_train[u]
            pos_batch = random.sample(pos_items, num)
            return pos_batch
        
        def sample_neg_items_for_u(u, num):
            neg_items = list(set(self.exist_items) - set(self.user_train[u]))
            neg_batch = random.sample(neg_items, num)
            return neg_batch
        
        pos_items, neg_items = [], []
        for user in users:
            pos_items += sample_pos_items_for_u(user, 1)
            neg_items += sample_neg_items_for_u(user, 1)
        
        return users, pos_items, neg_items

    def get_train_valid_data(self):
        return self.user_train, self.user_valid

    def get_R_data(self):
        return self.R_train, self.R_valid, self.R_total

    def get_ngcf_adj_matrix_data(self):
        return self.ngcf_adj_matrix

3. 모델

In [ ]:
class NGCF(nn.Module):
    def __init__(self, n_users, n_items, emb_dim, layers, reg, node_dropout, mess_dropout, adj_mtx):
        super().__init__()

        # initialize Class attributes
        self.n_users = n_users
        self.n_items = n_items
        self.emb_dim = emb_dim
        self.l_matrix = adj_mtx
        self.l_plus_i_matrix = adj_mtx + sp.eye(adj_mtx.shape[0])
        self.reg = reg
        self.layers = layers
        self.n_layers = len(self.layers)
        self.node_dropout = node_dropout
        self.mess_dropout = mess_dropout

        # Initialize weights
        self.weight_dict = self._init_weights()
        print("Weights initialized.")

        # Create Matrix 'L+I', PyTorch sparse tensor of SP adjacency_mtx
        self.L_plus_I = self._convert_sp_mat_to_sp_tensor(self.l_plus_i_matrix)
        self.L = self._convert_sp_mat_to_sp_tensor(self.l_matrix)

    # initialize weights
    def _init_weights(self):
        print("Initializing weights...")
        weight_dict = nn.ParameterDict()

        initializer = torch.nn.init.xavier_uniform_
        
        weight_dict['user_embedding'] = nn.Parameter(initializer(torch.empty(self.n_users, self.emb_dim).to(device)))
        weight_dict['item_embedding'] = nn.Parameter(initializer(torch.empty(self.n_items, self.emb_dim).to(device)))

        weight_size_list = [self.emb_dim] + self.layers

        for k in range(self.n_layers):
            weight_dict['W_one_%d' %k] = nn.Parameter(initializer(torch.empty(weight_size_list[k], weight_size_list[k+1]).to(device)))
            weight_dict['b_one_%d' %k] = nn.Parameter(initializer(torch.empty(1, weight_size_list[k+1]).to(device)))
            
            weight_dict['W_two_%d' %k] = nn.Parameter(initializer(torch.empty(weight_size_list[k], weight_size_list[k+1]).to(device)))
            weight_dict['b_two_%d' %k] = nn.Parameter(initializer(torch.empty(1, weight_size_list[k+1]).to(device)))
           
        return weight_dict

    # convert sparse matrix into sparse PyTorch tensor
    def _convert_sp_mat_to_sp_tensor(self, X):
        """
        Convert scipy sparse matrix to PyTorch sparse matrix

        Arguments:
        ----------
        X = Adjacency matrix, scipy sparse matrix
        """
        coo = X.tocoo().astype(np.float32)
        i = torch.LongTensor(np.mat([coo.row, coo.col]))
        v = torch.FloatTensor(coo.data)
        res = torch.sparse.FloatTensor(i, v, coo.shape).to(device)
        return res

    # apply node_dropout
    def _droupout_sparse(self, X):
        """
        Drop individual locations in X
        
        Arguments:
        ---------
        X = adjacency matrix (PyTorch sparse tensor)
        dropout = fraction of nodes to drop
        noise_shape = number of non non-zero entries of X
        """
        node_dropout_mask = ((self.node_dropout) + torch.rand(X._nnz())).floor().bool().to(device)
        i = X.coalesce().indices()
        v = X.coalesce()._values()
        i[:,node_dropout_mask] = 0
        v[node_dropout_mask] = 0
        X_dropout = torch.sparse.FloatTensor(i, v, X.shape).to(X.device)

        return  X_dropout.mul(1/(1-self.node_dropout))

    def forward(self, u, i, j):
        """
        Computes the forward pass
        
        Arguments:
        ---------
        u = user
        i = positive item (user interacted with item)
        j = negative item (user did not interact with item)
        """
        # apply drop-out mask
        L_plus_I_hat = self._droupout_sparse(self.L_plus_I) if self.node_dropout > 0 else self.L_plus_I
        L_hat = self._droupout_sparse(self.L) if self.node_dropout > 0 else self.L
        
        ego_embeddings = torch.cat([self.weight_dict['user_embedding'], self.weight_dict['item_embedding']], 0)

        final_embeddings = [ego_embeddings]

        for k in range(self.n_layers):
            
            # (L+I)E
            side_L_plus_I_embeddings = torch.sparse.mm(L_plus_I_hat, final_embeddings[k])
            
            # (L+I)EW_1 + b_1
            simple_embeddings = torch.matmul(side_L_plus_I_embeddings, self.weight_dict['W_one_%d' % k]) + self.weight_dict['b_one_%d' % k]
            
            # LE
            side_L_embeddings = torch.sparse.mm(L_hat, final_embeddings[k])                              
            
            # LEE
            interaction_embeddings = torch.mul(side_L_embeddings, final_embeddings[k])
                                             
            # LEEW_2 + b_2
            interaction_embeddings = torch.matmul(interaction_embeddings, self.weight_dict['W_two_%d' % k]) + self.weight_dict['b_two_%d' % k]

            # non-linear activation 
            ego_embeddings =  F.leaky_relu(simple_embeddings + interaction_embeddings)
            
            # add message dropout
            mess_dropout_mask = nn.Dropout(self.mess_dropout)
            ego_embeddings = mess_dropout_mask(ego_embeddings)

            # Perform L2 normalization
            norm_embeddings = F.normalize(ego_embeddings, p=2, dim=1)
            
            final_embeddings.append(norm_embeddings)                                            

        
        final_embeddings = torch.cat(final_embeddings, 1)                           
    
        # back to user/item dimension
        u_final_embeddings, i_final_embeddings = final_embeddings.split([self.n_users, self.n_items], 0)

        self.u_final_embeddings = nn.Parameter(u_final_embeddings)
        self.i_final_embeddings = nn.Parameter(i_final_embeddings)
        
        u_emb = u_final_embeddings[u] # user embeddings
        p_emb = i_final_embeddings[i] # positive item embeddings
        n_emb = i_final_embeddings[j] # negative item embeddings
    
        y_ui = torch.sum(torch.mul(u_emb, p_emb), dim = 1)                           
        y_uj = torch.sum(torch.mul(u_emb, n_emb), dim = 1)
        
        log_prob = torch.mean(torch.log(torch.sigmoid(y_ui - y_uj)))
        bpr_loss = -log_prob        
        if self.reg > 0.:
            l2norm = (torch.sum(u_emb**2)/2. + torch.sum(p_emb**2)/2. + torch.sum(n_emb**2)/2.) / u_emb.shape[0]
            l2reg = self.reg * l2norm
            bpr_loss += l2reg
        
        return bpr_loss

4. 학습 함수

In [ ]:
def train(model, make_graph_data_set, optimizer, n_batch):
    model.train()
    loss_val = 0
    for step in range(1, n_batch + 1):
        user, pos, neg = make_graph_data_set.sampling()
        optimizer.zero_grad()
        loss = model(user, pos, neg)
        loss.backward()
        optimizer.step()
        loss_val += loss.item()
    loss_val /= n_batch
    return loss_val

def split_matrix(X, n_splits=10):
    splits = []
    chunk_size = X.shape[0] // n_splits
    for i in range(n_splits):
        start = i * chunk_size
        end = X.shape[0] if i == n_splits - 1 else (i + 1) * chunk_size
        splits.append(X[start:end])
    return splits

def compute_ndcg_k(pred_items, test_items, test_indices, k):
    
    r = (test_items * pred_items).gather(1, test_indices)
    f = torch.from_numpy(np.log2(np.arange(2, k+2))).float().to(device)
    
    dcg = (r[:, :k]/f).sum(1)                                               
    dcg_max = (torch.sort(r, dim=1, descending=True)[0][:, :k]/f).sum(1)   
    ndcg = dcg/dcg_max                                                     
    
    ndcg[torch.isnan(ndcg)] = 0
    return ndcg

def evaluate(u_emb, i_emb, Rtr, Rte, k = 10):

    # split matrices
    ue_splits = split_matrix(u_emb)
    tr_splits = split_matrix(Rtr)
    te_splits = split_matrix(Rte)

    recall_k, ndcg_k= [], []
    # compute results for split matrices
    for ue_f, tr_f, te_f in zip(ue_splits, tr_splits, te_splits):

        scores = torch.mm(ue_f, i_emb.t())

        test_items = torch.from_numpy(te_f.todense()).float().to(device)
        non_train_items = torch.from_numpy(1-(tr_f.todense())).float().to(device)
        scores = scores * non_train_items

        _, test_indices = torch.topk(scores, dim=1, k=k)
        
        pred_items = torch.zeros_like(scores).float()
        pred_items.scatter_(dim=1, index=test_indices, src=torch.ones_like(test_indices).float().to(device))

        topk_preds = torch.zeros_like(scores).float()
        topk_preds.scatter_(dim=1, index=test_indices[:, :k], src=torch.ones_like(test_indices).float())
        
        TP = (test_items * topk_preds).sum(1)                      
        rec = TP/test_items.sum(1)
   
        ndcg = compute_ndcg_k(pred_items, test_items, test_indices, k)

        recall_k.append(rec)
        ndcg_k.append(ndcg)

    return torch.cat(ndcg_k).mean(), torch.cat(recall_k).mean()

5. 학습

In [ ]:
make_graph_data_set = MakeGraphDataSet(config = config)
ngcf_adj_matrix = make_graph_data_set.get_ngcf_adj_matrix_data()
R_train, R_valid, R_total = make_graph_data_set.get_R_data()

In [ ]:
model = NGCF(
    n_users = make_graph_data_set.num_user,
    n_items = make_graph_data_set.num_item,
    emb_dim = config.emb_dim,
    layers = config.layers,
    reg = config.reg,
    node_dropout = config.node_dropout,
    mess_dropout = config.mess_dropout,
    adj_mtx = ngcf_adj_matrix,
    ).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=config.lr)

Initializing weights...
Weights initialized.


In [ ]:
best_hit = 0
loss_list = []
ndcg_list = []
hit_list = []
for epoch in range(1, config.num_epochs + 1):
    tbar = tqdm(range(1))
    for _ in tbar:
        train_loss = train(
            model = model, 
            make_graph_data_set = make_graph_data_set, 
            optimizer = optimizer,
            n_batch = config.n_batch,
            )
        with torch.no_grad():
            ndcg, hit = evaluate(
                u_emb = model.u_final_embeddings.detach(), 
                i_emb = model.i_final_embeddings.detach(), 
                Rtr = R_train, 
                Rte = R_valid, 
                k = 10,
                )

        if best_hit < hit:
            best_hit = hit
            torch.save(model.state_dict(), os.path.join(config.model_path, config.model_name))
        
        
        loss_list.append(train_loss)
        ndcg_list.append(ndcg)
        hit_list.append(hit)

        tbar.set_description(f'Epoch: {epoch:3d}| Train loss: {train_loss:.5f}| NDCG@10: {ndcg:.5f}| HIT@10: {hit:.5f}')

Epoch:   1| Train loss: 0.71176| NDCG@10: 0.00694| HIT@10: 0.00153: 100%|██████████| 1/1 [00:06<00:00,  6.09s/it]
Epoch:   2| Train loss: 0.70834| NDCG@10: 0.00964| HIT@10: 0.00211: 100%|██████████| 1/1 [00:04<00:00,  4.65s/it]
Epoch:   3| Train loss: 0.70433| NDCG@10: 0.01177| HIT@10: 0.00271: 100%|██████████| 1/1 [00:04<00:00,  4.65s/it]
Epoch:   4| Train loss: 0.70324| NDCG@10: 0.01189| HIT@10: 0.00286: 100%|██████████| 1/1 [00:05<00:00,  5.10s/it]
Epoch:   5| Train loss: 0.69905| NDCG@10: 0.01276| HIT@10: 0.00281: 100%|██████████| 1/1 [00:04<00:00,  4.63s/it]
Epoch:   6| Train loss: 0.69851| NDCG@10: 0.01050| HIT@10: 0.00258: 100%|██████████| 1/1 [00:04<00:00,  4.65s/it]
Epoch:   7| Train loss: 0.69675| NDCG@10: 0.01369| HIT@10: 0.00310: 100%|██████████| 1/1 [00:05<00:00,  5.53s/it]
Epoch:   8| Train loss: 0.69575| NDCG@10: 0.01160| HIT@10: 0.00271: 100%|██████████| 1/1 [00:04<00:00,  4.60s/it]
Epoch:   9| Train loss: 0.69455| NDCG@10: 0.01972| HIT@10: 0.00439: 100%|██████████| 1/1

In [ ]:
# 마지막 50만 그림 그려보기
import matplotlib.pyplot as plt

fig, ax = plt.subplots(1, 3, figsize = (15, 5))
ax = ax.flatten()
epochs = [i for i in range(1, config.num_epochs + 1)]

ax[0].plot(epochs, loss_list)
ax[0].set_title('Loss')

ax[1].plot(epochs, ndcg_list)
ax[1].set_title('NDCG')

ax[2].plot(epochs, hit_list)
ax[2].set_title('HIT')
plt.show()